In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai.imports import *
from fastai.torch_imports import *
from fastai.core import *
from fastai.model import fit
from fastai.dataset import *

import torchtext
from torchtext import vocab, data
from torchtext.datasets import language_modeling

from fastai.rnn_reg import *
from fastai.rnn_train import *
from fastai.nlp import *
from fastai.lm_rnn import *
from fastai.plots import *

## Language modeling

### Data

In [2]:
PATH='data/wikitext-2/'
%ls {PATH}

models/  tmp/  wiki.test.tokens*  wiki.train.tokens*  wiki.valid.tokens*


In [3]:
!head -5 {PATH}wiki.train.tokens

 
 = Valkyria Chronicles III = 
 
 Senjō no Valkyria 3 : <unk> Chronicles ( Japanese : 戦場のヴァルキュリア3 , lit . Valkyria of the Battlefield 3 ) , commonly referred to as Valkyria Chronicles III outside Japan , is a tactical role @-@ playing video game developed by Sega and Media.Vision for the PlayStation Portable . Released in January 2011 in Japan , it is the third game in the Valkyria series . <unk> the same fusion of tactical and real @-@ time gameplay as its predecessors , the story runs parallel to the first game and follows the " Nameless " , a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " <unk> Raven " . 
 The game began development in 2010 , carrying over a large portion of the work done on Valkyria Chronicles II . While it retained the standard features of the series , it also underwent multiple adjustments , such as making the game more <unk> for series newcomers . Char

In [4]:
!wc -lwc {PATH}wiki.train.tokens

   36718  2051910 10797148 data/wikitext-2/wiki.train.tokens


In [5]:
!wc -lwc {PATH}wiki.valid.tokens

   3760  213886 1121681 data/wikitext-2/wiki.valid.tokens


In [6]:
TEXT = data.Field(lower=True)
FILES = dict(train='wiki.train.tokens', validation='wiki.valid.tokens', test='wiki.test.tokens')
bs,bptt = 80,70
md = LanguageModelData.from_text_files(PATH, TEXT, **FILES, bs=bs, bptt=bptt, min_freq=10)
len(md.trn_dl), md.nt, len(md.trn_ds), len(md.trn_ds[0].text)

(365, 12980, 1, 2051911)

In [7]:
#md.trn_ds[0].text[:12], next(iter(md.trn_dl))

### Train

In [8]:
emb_sz = 200
n_hid = 500
n_layers = 3
learner = md.get_model(opt_fn=SGD_Momentum(0.7), emb_sz=emb_sz, n_hid=n_hid, n_layers=n_layers)
reg_fn=partial(seq2seq_reg, alpha=2, beta=1)

In [9]:
clip=0.3
lr=10
cycles=1
weight_decay=1e-6
learner.reg_fn=reg_fn
learner.clip=clip
learner.fit(lrs=lr, n_cycle=cycles, wds=weight_decay)

epoch      trn_loss   val_loss                              
    0      6.147407   5.86342   



[array([5.86342])]

In [ ]:
cycles=6
learner.fit(lrs=lr, n_cycle=cycles, wds=weight_decay, cycle_len=1, cycle_mult=2)

epoch      trn_loss   val_loss                              
    0      5.910554   5.602705  
    1      5.784423   5.440854                              
    2      5.69788    5.377639                              
    3      5.634754   5.294885                              
    4      5.532404   5.193702                              
    5      5.461805   5.128325                              
    6      5.45189    5.115465                              
    7      5.440065   5.081868                              
    8      5.3455     5.024077                              
    9      5.282611   4.948805                              
    10     5.222102   4.89329                               
    11     5.18734    4.857407                              
    12     5.166218   4.838427                              
    13     5.133103   4.821393                              
    14     5.127364   4.817703                              
    15     5.172796   4.845742                      

In [ ]:
learner.save('lm_420')

In [ ]:
learner.load('lm_420')
learner.fit(lrs=lr, n_cycle=cycles, wds=weight_decay, cycle_len=1, cycle_mult=2)

In [ ]:
learner.save('lm_419')

In [ ]:
learner.load('lm_419')
learner.fit(lrs=lr, n_cycle=cycles, wds=weight_decay, cycle_len=1, cycle_mult=2)

In [ ]:
learner.save('lm_418')

In [ ]:
learner.load('lm_418')
math.exp(4.17)

### Test

In [ ]:
m=learner.model
s=[""". <eos> The game began development in 2010 , carrying over a large portion of the work 
done on Valkyria Chronicles II . While it retained the standard features of """.split()]
t=TEXT.numericalize(s)

m[0].bs=1
m.reset()
res,*_ = m(t)

In [ ]:
nexts = torch.topk(res[-1], 10)[1]
[TEXT.vocab.itos[o] for o in to_np(nexts)]

In [ ]:
for i in range(20):
    n=res[-1].topk(2)[1]
    n = n[1] if n.data[0]==0 else n[0]
    print(TEXT.vocab.itos[n.data[0]], end=' ')
    res,*_ = m(n[0].unsqueeze(0))

### End